In [1]:
import numpy as np
import rasterio
import pandas as pd
import os
import sys

In [2]:
# Get the current working directory
current_dir = os.path.abspath('')

# Search for the 'constants.py' file starting from the current directory and moving up the hierarchy
project_root = current_dir
while not os.path.isfile(os.path.join(project_root, 'constants.py')):
    project_root = os.path.dirname(project_root)

# Add the project root to the Python path
sys.path.append(project_root)



In [3]:
from constants import SERVER_PATH, OUTPUT_PATH, SIMULATION_FEATURES_DIR


In [4]:
SIMULATION_FEATURES_DIR

['/Users/romero61/../../capstone/pyforest/ml_data/output/sim_lup_features']

In [12]:
#output- update this for subsequent runs
output_folder = os.path.join(OUTPUT_PATH[0], 'predictions-log-lut-areas')
if not os.path.exists(output_folder):
    os.makedirs(output_folder)


In [9]:
sim25_lup = (os.path.join(SIMULATION_FEATURES_DIR[0],'sim25', "sim25_raster.tif"))
sim50_lup = (os.path.join(SIMULATION_FEATURES_DIR[0],'sim50', "sim50_raster.tif"))
sim5_lup = (os.path.join(SIMULATION_FEATURES_DIR[0],'sim5', "sim5_raster.tif"))
simhedges_lup = (os.path.join(SIMULATION_FEATURES_DIR[0],'simhedges', "simhedges_raster.tif"))


In [10]:
sim25_lup_prediction = os.path.join(OUTPUT_PATH[0], 'predictions-log', 'sim-25-all-feature-predicition.tiff')
sim50_lup_prediction = os.path.join(OUTPUT_PATH[0], 'predictions-log', 'sim-50-all-feature-predicition.tiff')
sim5_lup_prediction = os.path.join(OUTPUT_PATH[0], 'predictions-log', 'sim-5-all-feature-predicition.tiff')
simhedges_lup_prediction = os.path.join(OUTPUT_PATH[0], 'predictions-log', 'sim-hedges-all-feature-predicition.tiff')


# 25

In [20]:
# Open the land use raster file
with rasterio.open(sim25_lup) as src:
    land_use_array = src.read(1)

# Open the deforestation probability raster file
with rasterio.open(sim25_lup_prediction) as src:
    deforestation_probability_array = src.read(1)

# Mask the NoData values
land_use_array_masked = np.ma.masked_less(land_use_array, 0)

# Flatten the array
land_use_array_flat = land_use_array_masked.compressed()

# Count the number of pixels of each land use type
counts = np.bincount(land_use_array_flat)

# Calculate the area of each land use type
pixel_area = 900  # replace this with the actual area represented by each pixel
areas = counts * pixel_area

# Print the area of each land use type
for i, area in enumerate(areas):
    print(f"Area of land use type {i}: {area} square meters")
# Calculate the deforested area for each pixel
deforested_area_array = deforestation_probability_array * 900  # replace 900 with the actual area represented by each pixel

# Initialize arrays to hold the total deforested area, non-deforested area, and total area for each land use type
deforested_areas = np.zeros(4)  # number of land use types
non_deforested_areas = np.zeros(4)  # number of land use types
total_areas = np.zeros(4)  # number of land use types

# Loop over the land use types
for i in range(4):  # number of land use types
    # Select the deforested area for the pixels of the current land use type
    deforested_area = deforested_area_array[land_use_array == i+1]
    
    # Sum the deforested area
    deforested_areas[i] = deforested_area.sum()
    
    # Calculate the total area
    total_areas[i] = (land_use_array == i+1).sum() * pixel_area

    # Calculate the non-deforested area
    non_deforested_areas[i] = total_areas[i] - deforested_areas[i]
    # Calculate the area of each land use type in hectares
areas_hectares = areas / 10000  # 1 hectare = 10,000 square meters

# Print the area of each land use type in hectares
for i, area in enumerate(areas_hectares):
    print(f"Area of land use type {i}: {area} hectares")

# Calculate the deforested area for each pixel in hectares
deforested_area_array_hectares = deforestation_probability_array * pixel_area / 10000

# Initialize arrays to hold the total deforested area, non-deforested area, and total area for each land use type in hectares
deforested_areas_hectares = np.zeros(4)  # number of land use types
non_deforested_areas_hectares = np.zeros(4)  # number of land use types
total_areas_hectares = np.zeros(4)  # number of land use types

# Loop over the land use types
for i in range(4):  # number of land use types
    # Select the deforested area for the pixels of the current land use type
    deforested_area_hectares = deforested_area_array_hectares[land_use_array == i+1]
    
    # Sum the deforested area
    deforested_areas_hectares[i] = deforested_area_hectares.sum()
    
    # Calculate the total area in hectares
    total_areas_hectares[i] = (land_use_array == i+1).sum() * pixel_area / 10000

    # Calculate the non-deforested area in hectares
    non_deforested_areas_hectares[i] = total_areas_hectares[i] - deforested_areas_hectares[i]
    
# Create a DataFrame with the deforested area of each land use type in hectares
df_hectares = pd.DataFrame({
    'LandUseType': ['Hedgerow', 'Forest Reserve', 'Paddocks', 'Riparian Corridor'],
    'DeforestedArea': deforested_areas_hectares,
    'NonDeforestedArea': non_deforested_areas_hectares,
    'TotalArea': total_areas_hectares
})

output_file_hectares = os.path.join(output_folder, "sim-25-log-lut-area-hectares.csv")
# Export the DataFrame to a CSV file
df_hectares.to_csv(output_file_hectares, index=False)


Area of land use type 0: 0 square meters
Area of land use type 1: 13140840600 square meters
Area of land use type 2: 25707976200 square meters
Area of land use type 3: 59036355000 square meters
Area of land use type 4: 912870900 square meters
Area of land use type 0: 0.0 hectares
Area of land use type 1: 1314084.06 hectares
Area of land use type 2: 2570797.62 hectares
Area of land use type 3: 5903635.5 hectares
Area of land use type 4: 91287.09 hectares


# 50%

In [21]:
# Open the land use raster file
with rasterio.open(sim50_lup) as src:
    land_use_array = src.read(1)

# Open the deforestation probability raster file
with rasterio.open(sim50_lup_prediction) as src:
    deforestation_probability_array = src.read(1)

# Mask the NoData values
land_use_array_masked = np.ma.masked_less(land_use_array, 0)

# Flatten the array
land_use_array_flat = land_use_array_masked.compressed()

# Count the number of pixels of each land use type
counts = np.bincount(land_use_array_flat)

# Calculate the area of each land use type
pixel_area = 900  # replace this with the actual area represented by each pixel
areas = counts * pixel_area

# Print the area of each land use type
for i, area in enumerate(areas):
    print(f"Area of land use type {i}: {area} square meters")
# Calculate the deforested area for each pixel
deforested_area_array = deforestation_probability_array * 900  # replace 900 with the actual area represented by each pixel

# Initialize arrays to hold the total deforested area, non-deforested area, and total area for each land use type
deforested_areas = np.zeros(4)  # number of land use types
non_deforested_areas = np.zeros(4)  # number of land use types
total_areas = np.zeros(4)  # number of land use types

# Loop over the land use types
for i in range(4):  # number of land use types
    # Select the deforested area for the pixels of the current land use type
    deforested_area = deforested_area_array[land_use_array == i+1]
    
    # Sum the deforested area
    deforested_areas[i] = deforested_area.sum()
    
    # Calculate the total area
    total_areas[i] = (land_use_array == i+1).sum() * pixel_area

    # Calculate the non-deforested area
    non_deforested_areas[i] = total_areas[i] - deforested_areas[i]
    # Calculate the area of each land use type in hectares
areas_hectares = areas / 10000  # 1 hectare = 10,000 square meters

# Print the area of each land use type in hectares
for i, area in enumerate(areas_hectares):
    print(f"Area of land use type {i}: {area} hectares")

# Calculate the deforested area for each pixel in hectares
deforested_area_array_hectares = deforestation_probability_array * pixel_area / 10000

# Initialize arrays to hold the total deforested area, non-deforested area, and total area for each land use type in hectares
deforested_areas_hectares = np.zeros(4)  # number of land use types
non_deforested_areas_hectares = np.zeros(4)  # number of land use types
total_areas_hectares = np.zeros(4)  # number of land use types

# Loop over the land use types
for i in range(4):  # number of land use types
    # Select the deforested area for the pixels of the current land use type
    deforested_area_hectares = deforested_area_array_hectares[land_use_array == i+1]
    
    # Sum the deforested area
    deforested_areas_hectares[i] = deforested_area_hectares.sum()
    
    # Calculate the total area in hectares
    total_areas_hectares[i] = (land_use_array == i+1).sum() * pixel_area / 10000

    # Calculate the non-deforested area in hectares
    non_deforested_areas_hectares[i] = total_areas_hectares[i] - deforested_areas_hectares[i]
    
# Create a DataFrame with the deforested area of each land use type in hectares
df_hectares = pd.DataFrame({
    'LandUseType': ['Hedgerow', 'Forest Reserve', 'Paddocks', 'Riparian Corridor'],
    'DeforestedArea': deforested_areas_hectares,
    'NonDeforestedArea': non_deforested_areas_hectares,
    'TotalArea': total_areas_hectares
})

output_file_hectares = os.path.join(output_folder, "sim-50-log-lut-area-hectares.csv")
# Export the DataFrame to a CSV file
df_hectares.to_csv(output_file_hectares, index=False)

Area of land use type 0: 0 square meters
Area of land use type 1: 8532334800 square meters
Area of land use type 2: 50576823000 square meters
Area of land use type 3: 38781358200 square meters
Area of land use type 4: 912873600 square meters
Area of land use type 0: 0.0 hectares
Area of land use type 1: 853233.48 hectares
Area of land use type 2: 5057682.3 hectares
Area of land use type 3: 3878135.82 hectares
Area of land use type 4: 91287.36 hectares


In [22]:
# Open the land use raster file
with rasterio.open(sim5_lup) as src:
    land_use_array = src.read(1)

# Open the deforestation probability raster file
with rasterio.open(sim5_lup_prediction) as src:
    deforestation_probability_array = src.read(1)

# Mask the NoData values
land_use_array_masked = np.ma.masked_less(land_use_array, 0)

# Flatten the array
land_use_array_flat = land_use_array_masked.compressed()

# Count the number of pixels of each land use type
counts = np.bincount(land_use_array_flat)

# Calculate the area of each land use type
pixel_area = 900  # replace this with the actual area represented by each pixel
areas = counts * pixel_area

# Print the area of each land use type
for i, area in enumerate(areas):
    print(f"Area of land use type {i}: {area} square meters")
# Calculate the deforested area for each pixel
deforested_area_array = deforestation_probability_array * 900  # replace 900 with the actual area represented by each pixel

# Initialize arrays to hold the total deforested area, non-deforested area, and total area for each land use type
deforested_areas = np.zeros(4)  # number of land use types
non_deforested_areas = np.zeros(4)  # number of land use types
total_areas = np.zeros(4)  # number of land use types

# Loop over the land use types
for i in range(4):  # number of land use types
    # Select the deforested area for the pixels of the current land use type
    deforested_area = deforested_area_array[land_use_array == i+1]
    
    # Sum the deforested area
    deforested_areas[i] = deforested_area.sum()
    
    # Calculate the total area
    total_areas[i] = (land_use_array == i+1).sum() * pixel_area

    # Calculate the non-deforested area
    non_deforested_areas[i] = total_areas[i] - deforested_areas[i]
    # Calculate the area of each land use type in hectares
areas_hectares = areas / 10000  # 1 hectare = 10,000 square meters

# Print the area of each land use type in hectares
for i, area in enumerate(areas_hectares):
    print(f"Area of land use type {i}: {area} hectares")

# Calculate the deforested area for each pixel in hectares
deforested_area_array_hectares = deforestation_probability_array * pixel_area / 10000

# Initialize arrays to hold the total deforested area, non-deforested area, and total area for each land use type in hectares
deforested_areas_hectares = np.zeros(4)  # number of land use types
non_deforested_areas_hectares = np.zeros(4)  # number of land use types
total_areas_hectares = np.zeros(4)  # number of land use types

# Loop over the land use types
for i in range(4):  # number of land use types
    # Select the deforested area for the pixels of the current land use type
    deforested_area_hectares = deforested_area_array_hectares[land_use_array == i+1]
    
    # Sum the deforested area
    deforested_areas_hectares[i] = deforested_area_hectares.sum()
    
    # Calculate the total area in hectares
    total_areas_hectares[i] = (land_use_array == i+1).sum() * pixel_area / 10000

    # Calculate the non-deforested area in hectares
    non_deforested_areas_hectares[i] = total_areas_hectares[i] - deforested_areas_hectares[i]
    
# Create a DataFrame with the deforested area of each land use type in hectares
df_hectares = pd.DataFrame({
    'LandUseType': ['Hedgerow', 'Forest Reserve', 'Paddocks', 'Riparian Corridor'],
    'DeforestedArea': deforested_areas_hectares,
    'NonDeforestedArea': non_deforested_areas_hectares,
    'TotalArea': total_areas_hectares
})

output_file_hectares = os.path.join(output_folder, "sim-5-log-lut-area-hectares.csv")
# Export the DataFrame to a CSV file
df_hectares.to_csv(output_file_hectares, index=False)

Area of land use type 0: 0 square meters
Area of land use type 1: 17575208100 square meters
Area of land use type 2: 5497432200 square meters
Area of land use type 3: 74801823300 square meters
Area of land use type 4: 912858300 square meters
Area of land use type 0: 0.0 hectares
Area of land use type 1: 1757520.81 hectares
Area of land use type 2: 549743.22 hectares
Area of land use type 3: 7480182.33 hectares
Area of land use type 4: 91285.83 hectares
